In [1]:
#https://motius.breezy.hr/p/0b622aa1163e01-join-our-tech-community-as-working-student-m-f-d/apply?utm_source=pt-linkedin&utm_medium=job-posting

In [2]:
"""!pip install datasets
!pip install transformers
!pip install sentence_transformers
!pip install peft
!pip install huggingface_hub
!pip install --upgrade accelerate
!pip install --upgrade bitsandbytes
!pip install --upgrade torchmetrics
!pip install --upgrade deepspeed
"""

'!pip install datasets\n!pip install transformers\n!pip install sentence_transformers\n!pip install peft\n!pip install huggingface_hub\n!pip install --upgrade accelerate\n!pip install --upgrade bitsandbytes\n!pip install --upgrade torchmetrics\n!pip install --upgrade deepspeed\n'

In [1]:
from transformers import  AutoModel,AutoTokenizer,AutoModelForMaskedLM , Trainer,TrainingArguments,\
BitsAndBytesConfig,pipeline,default_data_collator,DataCollatorWithPadding,DataCollatorForLanguageModeling
from transformers.utils import move_cache
#from chat_template_utils import get_json_schema
#from llama_cpp import Llama
from utils import *
from peft import *
import datasets
#import torchmetrics
import torch
from torch.utils.data.dataloader import DataLoader,Dataset
#from hqq.engine.hf import HQQModelForCausalLM
#from hqq.models.hf.base import AutoHQQHFModel
from huggingface_hub import snapshot_download
#import deepspeed
import os
import pandas as pd
import json
#from var_dump import var_dump
from bs4 import BeautifulSoup,PageElement,Comment
import matplotlib.pyplot as plt
import seaborn as sns

cache_dir='/var/tmp/.cache/' #'/proj/ciptmp/ix05ogym/.cache/'
output_dir = cache_dir+'outputs/'

move_cache(cache_dir)

def pd_full_screen():
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    
def pd_normal_screen():
    pd.set_option('display.max_rows', 10)
    pd.set_option('display.max_columns', 5)


No ROCm runtime is found, using ROCM_HOME='/usr'
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


0it [00:00, ?it/s]

In [48]:
with open('result.json','r',encoding='utf-8') as file:
    
    j = file.read()
    j = json.loads(j)
    print(j.keys())
    

dict_keys(['name', 'type', 'id', 'messages'])


In [139]:
#j['messages'][1]['from']
#datasets.Dataset.from_json('result.json',field="messages")
df = pd.read_json('result.json')


info_df = pd.json_normalize(df['messages'])[['from','text']]

#joined = info_df['from']+': '+info_df['text']
info_df = info_df[~(info_df['text']=='')].dropna().reset_index(drop=True)
#info_df[~(info_df['from']=='')]

info_df

,from,text
0,مجید,خب من دیروز ساعت 9وربع رسیدم خونه :)
1,مسعود شرقی,Salam
2,مسعود شرقی,Hfhgghg
3,مسعود شرقی,Hfgghfggfggggggfgv
4,مجید,Hh
...,...,...
32501,مسعود شرقی,این چتها که فرستادی رو میگی؟
32502,مسعود شرقی,اخرش گفتی دوتا ویدئو فرستاد و پاک کرد من تا هم...
32503,مجید,ولش کون
32504,مسعود شرقی,اره خوندم خوب بود نکته ای نبود که بخام بگم.همی...


In [152]:
c = info_df['text'].str.contains("\[",na=True)
x = info_df.loc[c,:].reset_index()
x

,index,from,text
0,7,مجید,[بخشی از سهام شرکت سرمایه‌گذاری «دیجی‌کالا» و ...
1,12,مسعود شرقی,[پس از مدت ها سعی و تلاش شبانه روزی توانستیم ت...
2,13,مسعود شرقی,[پس از مدت ها سعی و تلاش شبانه روزی توانستیم ت...
3,30,مسعود شرقی,"[{'type': 'link', 'text': 'http://btab.ir/'}]"
4,35,مجید,[💢 گیف شمارۀ 116:\nچطور تخم مرغ رو بندازیم توی...
...,...,...,...
653,31505,مجید,"[{'type': 'link', 'text': 'https://youtube.com..."
654,31606,مجید,"[{'type': 'link', 'text': 'https://youtube.com..."
655,31731,مجید,"[{'type': 'custom_emoji', 'text': '💎', 'docume..."
656,31798,مجید,"[{'type': 'link', 'text': 'https://autoapply.j..."


In [155]:
print(x.loc[0,'text'])

['بخشی از سهام شرکت سرمایه\u200cگذاری «دیجی\u200cکالا» و «کافه بازار»  به ارزش حدود ۶ میلیون و ۴۰۰ هزار یورو توسط یک «شرکتی سوئدی» خریداری شده است.\n', {'type': 'mention', 'text': '@manototv'}, '']


In [85]:
c1 = info_df['from']=='مسعود شرقی'
c2 = info_df['from'] == 'مجید'
info_df[~( c1 | c2 )]

,from,text


In [165]:
def get_one_msg(x):
    text = x['text']
    if not isinstance(text,str):
        #print(text[0])
        return pd.NA
    
    return  { 
     "role":x['from'],
     "content": text
    }
    
data = info_df.apply(get_one_msg,axis=1)
data = data.dropna()
data

0        {'role': 'مجید', 'content': 'خب من دیروز ساعت ...
1               {'role': 'مسعود شرقی', 'content': 'Salam'}
2             {'role': 'مسعود شرقی', 'content': 'Hfhgghg'}
3        {'role': 'مسعود شرقی', 'content': 'Hfgghfggfgg...
4                        {'role': 'مجید', 'content': 'Hh'}
                               ...                        
32501    {'role': 'مسعود شرقی', 'content': 'این چتها که...
32502    {'role': 'مسعود شرقی', 'content': 'اخرش گفتی د...
32503               {'role': 'مجید', 'content': 'ولش کون'}
32504    {'role': 'مسعود شرقی', 'content': 'اره خوندم خ...
32505    {'role': 'مسعود شرقی', 'content': 'حالا یه کم ...
Length: 31853, dtype: object

In [ ]:
chat_template="""
{user1}: {user1_text}
{user2}: {user2_text}
"""

In [166]:
tokenizer.apply_chat_template()

[{'role': 'system', 'content': 'زبان شما فارسی است.'},
 {'role': 'مجید', 'content': 'خب من دیروز ساعت 9وربع رسیدم خونه :)'},
 {'role': 'مسعود شرقی', 'content': 'Salam'},
 {'role': 'مسعود شرقی', 'content': 'Hfhgghg'},
 {'role': 'مسعود شرقی', 'content': 'Hfgghfggfggggggfgv'},
 {'role': 'مجید', 'content': 'Hh'}]

In [5]:
#torch.cuda.get_device_name()

In [4]:
train_dataset = pd.read_excel('train_dataset.xlsx',index_col=0)
#train_dataset = train_dataset.drop_duplicates()
#train_dataset.to_excel('train_dataset.xlsx')
train_dataset


,query,element,label,action,utterance
0,Email Address,"<input id=""fieldEmail"" name=""cm-ukutgu-ukutgu""...",email,NaN,NaN
1,Accept additional cookies,"<input id=""accept_all_cookies_button"" messaged...",cookie,NaN,NaN
2,Reject additional cookies,"<input id=""accept_essential_cookies_button"" me...",cookie,NaN,NaN
3,Your Name,"<input aria-describedby=""ph-a11y-email-job-nam...",first name,NaN,NaN
4,Recipient’s Email address,"<input aria-describedby="""" aria-label=""Please ...",email,NaN,NaN
...,...,...,...,...,...
2716,og in,"<button class=""careersite-button careersite-bu...",login,1.0,log in
2717,upload resume,"<a href=""javascript: void(0);"" title=""upload r...",cv,1.0,upload resume
2718,upload fileprivacy policy,"<a href=""https://vertriebskarriere.hansemerkur...",privacy,1.0,privacy policy
2719,back,"<a href=""https://bewerber-innendienst.hansemer...",cancel,1.0,back


In [5]:
train_dataset[train_dataset['element'].isna()]

,query,element,label,action,utterance


In [6]:
test_dataset = pd.read_excel('./data/forms.xlsx')
c = test_dataset['label'].notna()
test_dataset = test_dataset.loc[c,:].reset_index(drop=True)
test_dataset

,query,element,label,sudo_label
0,Personal information\n CV or resume\n Browse C...,"<input accept=""application/pdf,.pdf"" name=""cv""...",cv,login
1,Name *\n,"<input name=""first_name"" placeholder=""First"" r...",first name,first name
2,This field contains invalid characters\n,"<input name=""last_name"" placeholder=""Last"" req...",last name,transcript
3,This field contains invalid characters\n Email...,"<input name=""email"" placeholder=""yourmail@doma...",email,email
4,Phone\n,"<input name=""phone"" placeholder=""+49 176 123 4...",phone number,phone number
...,...,...,...,...
95,cover letter\n upload your cover letter\n,"<input accept=""application/msword, application...",cover letter,NaN
96,github\n,"<input name=""custom_attribute_356304"" placehol...",github,NaN
97,ihre e-mail-adresse\n,"<button aria-label=""subscribe for newsletter"" ...",newsletter,NaN
98,(!)\n this field is required\n,"<button id=""uploaddropboxresume"" type=""submit""...",submit,NaN


In [7]:

person = {
    "first name": "majid",
    "last name": "sharghi foroushani",
    "pronoun": "he",
    "birth": "10/08/1997",
    "age": "27",
    "sex": "male",
    "skill": "NLP,LLM",
    "country": "Germany",
    "address": "Philip-9",
    "city": "Erlangen",
    "phone number": "+497773154",
    "available from": "6/24/2024",
    "salary": "10000",
    "German language level": "A1",
    "English language level": "C1",
    "visa": "I do not need visa.",
    "email": "sharghi.majid@gmail.com",
    "password": "123456",
    "find us": "Google",
    "xing": "www.xing.com",
    "job title": "I am a stduent.",
    "university": "FAU",
    "major": "AI",
    "privacy": "privacy",
    "search": "search",
    "submit": "submit",
    "cookie": "cookie",
    "cover letter": "cover letter",
    "linkedin": "www.linkedin.com",
    "github": "www.github.com",
    "login": "login",
    "register": "register",
    "cv": "/pah/to/cv.pdf",
    "twitter": "www.twitter.com",
    "years of experience": "4 years in deep learning",
    "username":"Majidsh97",
    "website language":"english",
    "transcript":"path/to/ts.pdf",
    "company":"company",
    "house number":"9",
    "italian language level":"A2",
    "ethnicity":"white",
    "picture":"path/to/p.jpg",
    "career level":"entry",
    "zip":"91052",
    "newsletter":"newsletter",
    "fake":"fake",
    "question":"question",
    "cancel":"cancel",
    "recommender":"my recommender",
    "website":"my website",
    'workherebefore':"No",
    "next":"next",
    "dropbox":"www.dropbox.com",
    "googledrive":"googledrive",
    "work hours":"8",
    "workduration":"6 months",
    "willing to relocate":"yes",
    "universityenrollment":"path to enrollment letter",
    "subscribe":"subscribe",
    "distance":"50 km",
    "captcha":"captcha",
    "forgot password":"no",
    "graduation year":"2024",
    "civic number":"54864",
    "person title":"Mr",
    "show password":"show password",
    
    
}
# {'years of experience', 'forgot password', 'next', 'username', 'website language', 'latitude', 'distance', 'transcript', 'english language level', 'cancel', 'company', 'longitude', 'house number', 'italian language level', 'title', 'captcha', 'civic number', 'zip', 'recommender', 'newsletter', 'show password', 'ethnicity', 'graduation year', 'job title', 'subscribe', 'picture', 'career level'}




In [90]:
from transformers import Qwen2ForCausalLM,T5ForConditionalGeneration
device = 'cuda'
model_name = 'Qwen/Qwen2-0.5B-Instruct'
#model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir=cache_dir)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:

model = Qwen2ForCausalLM.from_pretrained(model_name #Qwen/Qwen2-0.5B-Instruct
                                         #,attn_implementation="flash_attention_2"
                                         #,torch_dtype=torch.float16
                                         , device_map='auto',cache_dir=cache_dir).to('cuda')

model.eval()
#model.config['use_sliding_window']=True
print(model.config)

model

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Qwen2Config {
  "_name_or_path": "Qwen/Qwen2-0.5B-Instruct",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "max_position_embeddings": 32768,
  "max_window_layers": 24,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1e-06,
  "rope_theta": 1000000.0,
  "sliding_window": 32768,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.41.2",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}



Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Linear(in_featur

In [167]:
windows_size = 5 
train_dataset = []
for i in range(len(data)-windows_size):
    d = [{"role":"system","content":"زبان شما فارسی است."}]
    d.extend( data[i:i+windows_size].tolist() )
    train_dataset.append(d)
    
train_dataset

[[{'role': 'system', 'content': 'زبان شما فارسی است.'},
  {'role': 'مجید', 'content': 'خب من دیروز ساعت 9وربع رسیدم خونه :)'},
  {'role': 'مسعود شرقی', 'content': 'Salam'},
  {'role': 'مسعود شرقی', 'content': 'Hfhgghg'},
  {'role': 'مسعود شرقی', 'content': 'Hfgghfggfggggggfgv'},
  {'role': 'مجید', 'content': 'Hh'}],
 [{'role': 'system', 'content': 'زبان شما فارسی است.'},
  {'role': 'مسعود شرقی', 'content': 'Salam'},
  {'role': 'مسعود شرقی', 'content': 'Hfhgghg'},
  {'role': 'مسعود شرقی', 'content': 'Hfgghfggfggggggfgv'},
  {'role': 'مجید', 'content': 'Hh'},
  {'role': 'مسعود شرقی', 'content': 'Hggg'}],
 [{'role': 'system', 'content': 'زبان شما فارسی است.'},
  {'role': 'مسعود شرقی', 'content': 'Hfhgghg'},
  {'role': 'مسعود شرقی', 'content': 'Hfgghfggfggggggfgv'},
  {'role': 'مجید', 'content': 'Hh'},
  {'role': 'مسعود شرقی', 'content': 'Hggg'},
  {'role': 'مجید', 'content': ':|'}],
 [{'role': 'system', 'content': 'زبان شما فارسی است.'},
  {'role': 'مسعود شرقی', 'content': 'Hfgghfggfggggg

In [169]:
tokenized_train_data= tokenizer.apply_chat_template(train_dataset)


In [171]:
tokenizer.decode( tokenized_train_data[0] )

'<|im_start|>system\nزبان شما فارسی است.<|im_end|>\n<|im_start|>مجید\nخب من دیروز ساعت 9وربع رسیدم خونه :)<|im_end|>\n<|im_start|>مسعود شرقی\nSalam<|im_end|>\n<|im_start|>مسعود شرقی\nHfhgghg<|im_end|>\n<|im_start|>مسعود شرقی\nHfgghfggfggggggfgv<|im_end|>\n<|im_start|>مجید\nHh<|im_end|>\n'

In [26]:
s="""به فارسی بگو چند سالته"""
x = tokenizer(s,return_tensors='pt').to('cuda')

g=model.generate(inputs=x['input_ids'],max_new_tokens=200)

x = tokenizer.decode(g[0])

print(x)

به فارسی بگو چند سالته را در مورد دوست خود که در تجربه های مختلف، جذاب و نادیده باشند، ارائه دهم.
1. "دوست من: شما پرسنل بدنی" - به طور کامل برای کسب اطلاعات متفاوت و شخصیتمند داشته باشد، ولی نمی‌توانید از آنها بپرسید.
2. "دوست من: شما پرسنل راهنمایی" - این مقاله به دنبال ارزش مفید و روزمره از زمان باکتری، برای کسب اطلاعات متفاوت و شخصیتمند داشته باشد.
3. "دوست من: شما پرسنل روندی" - در


In [9]:
model.generation_config

GenerationConfig {
  "bos_token_id": 151643,
  "do_sample": true,
  "eos_token_id": [
    151645,
    151643
  ],
  "pad_token_id": 151643,
  "repetition_penalty": 1.1,
  "temperature": 0.7,
  "top_k": 20,
  "top_p": 0.8
}

In [10]:
model = get_peft_model(model,LoraConfig(task_type=TaskType.CAUSAL_LM,target_modules=['q_proj','v_proj','k_proj']))
#model = get_peft_model(model,LoraConfig(task_type=TaskType.CAUSAL_LM,target_modules=['q','v','k']))

model.print_trainable_parameters()
model
for n,p in model.named_parameters():
    if p.requires_grad==False:
        print(n)

trainable params: 737,280 || all params: 494,770,048 || trainable%: 0.1490
base_model.model.model.embed_tokens.weight
base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight
base_model.model.model.layers.0.self_attn.q_proj.base_layer.bias
base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight
base_model.model.model.layers.0.self_attn.k_proj.base_layer.bias
base_model.model.model.layers.0.self_attn.v_proj.base_layer.weight
base_model.model.model.layers.0.self_attn.v_proj.base_layer.bias
base_model.model.model.layers.0.self_attn.o_proj.weight
base_model.model.model.layers.0.mlp.gate_proj.weight
base_model.model.model.layers.0.mlp.up_proj.weight
base_model.model.model.layers.0.mlp.down_proj.weight
base_model.model.model.layers.0.input_layernorm.weight
base_model.model.model.layers.0.post_attention_layernorm.weight
base_model.model.model.layers.1.self_attn.q_proj.base_layer.weight
base_model.model.model.layers.1.self_attn.q_proj.base_layer.bias
base_model.model.model

In [11]:
import random


def get_random():
    d = {}
    for k in person.keys():
        d[k] = str(random.randint(1000, 10000))

    return d


# get_random()
def dict_to_str(p):
    s = "{"
    for k, v in p.items():
        s += f'"{k}":"{v}",'
        pass

    s += "}"
    return s


def get_one_chat_template(query,element,utterance="",person_str="", label_str=None,label_action=None,action_history=""):
    d = [
        {   #{"role":"system","content":""}
            "role": "user",
            #"content": """Your answer has to be in json format like {"answer":""}. Use this content in your answer:\n"""
            "content": """Your answer has to be in short JSON format. Use this CONTENT in your answer:\n"""+
            f"""CONTENT: {{"{utterance}",{person_str}}}."""

            #f"\ncontent: {person_str}"
            
        },
        {
            "role": "user",
            #"content": "What is this HTML element about?\n" + query_element_str,
            #"content": "Question:\n" + query_element_str,
            
            "content": f"QUERY: {query}"+
            f"\nHTML: {element}"
            #f"\nutterance: {utterance}"
            
            
        },
    ]
    if label_str:
        d.append(
            {"role": "assistant", "content": '{"answer":"' +str( label_str )+ '","click":"'+str(label_action)+'"}'},
        )
        
    return d


def get_one_msg(text, label):
    p = person #get_random()
    strp = ""#dict_to_str(p)
    rc = ""  # random.choice(list(p.keys()))+' '
    d = get_one_chat_template(strp, rc + text,label)#p[label])
    return d


def get_chat():

    d=train_dataset.apply( lambda x :get_one_chat_template(x['query'],x['element'],x['utterance'],"",x['label'],x['action']),axis=1).tolist()

    return d  # ,labels


get_chat()

[[{'role': 'user',
   'content': 'Your answer has to be in short JSON format. Use this CONTENT in your answer:\nCONTENT: {"nan",}.'},
  {'role': 'user',
   'content': 'QUERY: Email Address\nHTML: <input id="fieldEmail" name="cm-ukutgu-ukutgu" required="" type="email"/>'},
  {'role': 'assistant', 'content': '{"answer":"email","click":"nan"}'}],
 [{'role': 'user',
   'content': 'Your answer has to be in short JSON format. Use this CONTENT in your answer:\nCONTENT: {"nan",}.'},
  {'role': 'user',
   'content': 'QUERY: Accept additional cookies\nHTML: <input id="accept_all_cookies_button" messagedays="90" name="accept_all_cookies_button" type="submit" value="Accept additional cookies"/>'},
  {'role': 'assistant', 'content': '{"answer":"cookie","click":"nan"}'}],
 [{'role': 'user',
   'content': 'Your answer has to be in short JSON format. Use this CONTENT in your answer:\nCONTENT: {"nan",}.'},
  {'role': 'user',
   'content': 'QUERY: Reject additional cookies\nHTML: <input id="accept_essen

In [12]:
def get_train_data():
    return tokenizer.apply_chat_template(get_chat(),add_generation_prompt=False ,truncation=True,max_length=512)  # ,return_tensors='pt')
    pass

tokenized_data = get_train_data()


no_label = tokenizer.apply_chat_template([  get_one_chat_template( 
                                                                   str(x['query']),
                                                                   x['element']
                                                                   ) 
                                for i,x in test_dataset.iterrows()] ,truncation=True,max_length=512)  # ,return_tensors='pt')

print(len(tokenized_data))
print(tokenizer.decode(token_ids=tokenized_data[0]))
print(len(tokenized_data[0]))
print(tokenizer.decode(token_ids=no_label[0]))


2721
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Your answer has to be in short JSON format. Use this CONTENT in your answer:
CONTENT: {"nan",}.<|im_end|>
<|im_start|>user
QUERY: Email Address
HTML: <input id="fieldEmail" name="cm-ukutgu-ukutgu" required="" type="email"/><|im_end|>
<|im_start|>assistant
{"answer":"email","click":"nan"}<|im_end|>

90
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Your answer has to be in short JSON format. Use this CONTENT in your answer:
CONTENT: {"",}.<|im_end|>
<|im_start|>user
QUERY: Personal information
 CV or resume
 Browse CV to autocomplete your application. You can still fill your profile manually.
 
HTML: <input accept="application/pdf,.pdf" name="cv" tabindex="-1" type="file"/><|im_end|>



In [13]:
len(tokenized_data[0])

90

In [14]:
#tokenizer(['<|im_start|>assistant'])
tokenizer(['assistant'])


{'input_ids': [[77091]], 'attention_mask': [[1]]}

In [15]:
mydc = DataCollatorForLanguageModeling(tokenizer,False)
def mmf(x):
    y = mydc(x)
    b = (y['labels']==77091).int().argmax()-1
    for i in range(0,b):
        y['labels'][0][i]=-100
        pass
    #print( y['labels'] )
    return y



In [16]:
from transformers import TrainerCallback
"""
class ChangeDatasetCallback(TrainerCallback):
    def __init__(self):
        #super.__init__()
        pass

    def on_epoch_end(self, args, state, control, **kwargs):
        # Move to the next dataset at the end of each epoch
        #self.current_epoch += 1
        #new_dataset = self.datasets[self.current_epoch % len(self.datasets)]
        kwargs['train_dataloader'].dataset = get_train_data()
        #kwargs['eval_dataloader'].dataset = new_dataset['validation']
"""

class custom_dataset(Dataset):
    def __init__(self) -> None:
        super().__init__()
        self.dataset = get_train_data()
        self.len = len(self.dataset)
        pass

    def __getitem__(self, i) :
        if i==self.len-1:
            self.dataset = get_train_data()

        return self.dataset[i]
        pass

    def __len__(self):
        return self.len
        pass

trainer =  Trainer(model,
        train_dataset=custom_dataset(),
        data_collator=mmf,
        #compute_metrics=compute_metric,
        #callbacks=[ChangeDatasetCallback()],
        args=TrainingArguments(
          output_dir=output_dir,
          num_train_epochs=2,
          per_device_train_batch_size=1,
          per_device_eval_batch_size=1,
          logging_steps=300,

          #eval_accumulation_steps =2,


        ))

trainer.train()
model = model.eval()

[2024-07-08 11:17:19,627] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible


Step,Training Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 298.00 MiB. GPU 

In [ ]:
#trainer.model.save_pretrained(output_dir+'saved_84/')


/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
questions = [
"use html to create email"
,"""what is your gender?"""
,"""where are you living?(optional)"""
,"what is your salary expectation?*"
,"Which pronouns should we use to address you? (optional)"
,"how is your German?"
,"do you need visa spunsership?"
,"where are you working right now?"
,"what is your birth date?"
,"what are your hobbies?"
,"password"
,"what projects have you done?"
,"are you a text ai?"
,"what experice do you have in network engineering?"
,"what projects have you done in deep learning?"
,"how did you find us?"]

#model.disable_adapter()

x= [get_one_chat_template(""#person_to_str(person)
                          ,q+"""<input type="text" style="" id="custom">""") for q in questions]
tokenized_questions = tokenizer.apply_chat_template(x)

with torch.no_grad():
    
    for i,x in enumerate(tokenized_questions):
            x = torch.tensor([x]).to('cuda')
            o = model.generate(input_ids=x,max_new_tokens=500,do_sample=False)
            
            print(questions[i])
            #decoded = tokenizer.decode(o[0,x.shape[1]+3:])
            decoded = tokenizer.decode(o[0])
            
            print(decoded)
            print('--------------------------------------------------------')
        
    

/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:537: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


use html to create email
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Your answer has to be in short JSON format. Use this CONTENT in your answer:
CONTENT: {"",}.<|im_end|>
<|im_start|>user
QUERY: 
HTML: use html to create email<input type="text" style="" id="custom"><|im_end|>
<|im_start|>assistant
{"answer":"<html><body> <input type="text" style="" id="custom"></body></html>"}<|im_end|>
--------------------------------------------------------
what is your gender?
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Your answer has to be in short JSON format. Use this CONTENT in your answer:
CONTENT: {"",}.<|im_end|>
<|im_start|>user
QUERY: 
HTML: what is your gender?<input type="text" style="" id="custom"><|im_end|>
<|im_start|>assistant
{"answer":"female"}<|im_end|>
--------------------------------------------------------
where are you living?(optional)
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Your

In [ ]:
model = model.to('cuda')
def get_answer(decoded):
    try:
                #print("ERROR: ",decoded)

                _s = decoded.find('{')
                _e = decoded.find('}',_s)+1
                #print(decoded[_s:_e])
                answer = json.loads(decoded[_s:_e])['answer']
                return answer
                
    except Exception as e:
                print("ERROR: ",e)
                print(decoded)
                print("---------------------------------")

                
def compute_metric():
    model.eval()
    acc=0
    with torch.no_grad():
        for i in range(len(no_label)):
            x = mydc([no_label[i]])['input_ids'].to('cuda')
            #print(x)
            o = model.generate(input_ids=x,max_new_tokens=20,do_sample=False)
            decoded = tokenizer.decode(o[0,x.shape[1]:])
            #print(decoded)
            #break

            #print(x[0],x[1])
            answer = get_answer(decoded)
            _label = test_dataset.loc[i,'label']
            #_label = person[test_dataset.loc[i,'label']]
            c = answer==_label

            if c:
                acc +=1
            else:
                print("NO_CORRECT:",answer,_label)
                print("---------------------------------")

                pass

    acc/=len(no_label)
    print("acc:",acc)



compute_metric()

NO_CORRECT: browseCV cv
---------------------------------
NO_CORRECT: John first name
---------------------------------
ERROR:  'answer'
<|im_start|>assistant
{"error": "Invalid character in input field"}<|im_end|>
---------------------------------
NO_CORRECT: None last name
---------------------------------
ERROR:  'answer'
<|im_start|>assistant
{"error": "Invalid character in email address"}<|im_end|>
---------------------------------
NO_CORRECT: None email
---------------------------------
ERROR:  Expecting value: line 1 column 1 (char 0)
<|im_start|>assistant
{"answer":"+49 176 123 44
---------------------------------
NO_CORRECT: None phone number
---------------------------------
NO_CORRECT: 2021-01-01 available from
---------------------------------
ERROR:  Expecting value: line 1 column 1 (char 0)
<|im_start|>assistant
{"answer":"https://github.com/","icon":"https://assets.ccbp
---------------------------------
NO_CORRECT: None github
---------------------------------
ERROR:  Ex

In [ ]:
model.eval()
i =11
d = mydc(no_label[i:i+1])['input_ids'].to(device)
d = tokenizer.apply_chat_template( [#{"role":"system","content":""}
                             {"role":"user", "content":"""Your answer has to be in json format like {"answer":""}. Use this content in your answer:\n"""+s}
                            ,{"role":"user", "content":'email'}] , return_tensors='pt').to(device)
#d = tokenizer(['json {"answer":""} what is this html element about? <input type="email">'],return_tensors='pt')['input_ids'].to(device)
#print(d)
with torch.no_grad():

    o = model.generate(input_ids = d,max_length = 512,do_sample=True)
    print(tokenizer.decode(o[0]))



<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Your answer has to be in json format like {"answer":""}. Use this content in your answer:
["birth","major","cookie","search","city","English language level","years of experience","forgot password","next","username","address","website language","latitude","distance","cv","phone number","transcript","english language level","register","pronoun","university","cancel","job","company","longitude","age","house number","skill","italian language level","visa","title","twitter","captcha","civic number","zip","find us","recommender","available from","linkedin","submit","email","salary","newsletter","first name","German language level","show password","last name","ethnicity","github","cover letter","graduation year","sex","password","privacy","login","job title","subscribe","picture","country","xing","career level",]<|im_end|>
<|im_start|>user
email<|im_end|>
<|im_start|>assistant
{"answer":"johndoe@example.com"}<|im_end|>

In [ ]:
"""x =Qwen2ForCausalLM.from_pretrained('Qwen/Qwen2-0.5B-Instruct',cache_dir=cache_dir)
model = PeftModelForCausalLM.from_pretrained(x,output_dir+'saved/')"""